In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
if '..' not in sys.path: sys.path.append('..')
from f_ring_util import (compute_corrected_ew, 
                         compute_corrected_ew_col,
                         fit_hg_phase_function, 
                         hg_func,
                         print_hg_params)
pd.set_option('display.max_rows', None)

# analyze_tau version that only applies tau correction to core range (but, includes all points in optimization)
# (cleaned-up version of 3-region analyze_tau script, also using slope tau optimization)

In [2]:
# Define functions to process data

def remove_eqx(orig_obsdata, eqx_cutoff):
    #remove EQX observations
    #orig_obsdata: original pandas dataframe
    #returns: obsdata (EQX observations removed)
    
    good_i = np.abs(orig_obsdata['Incidence']-90) > eqx_cutoff
    obsdata = orig_obsdata[good_i]
    print('Removed EQX OBSIDs:', set(orig_obsdata[~good_i].groupby('Observation').indices))
    print('Unique observation names:', len(obsdata))
    print()
    
    return obsdata

def remove_outliers(obsdata, outlier_obs):
    #remove 3 outliers from ~phase 50-60 before fitting for tau
    #(located by hand, removing from all 3 by observation name)
    
    for obsid in outlier_obs:
        #drop row containing outlier ob
        obsdata = obsdata[obsdata.index != obsid]
            
    print('Removed outlier OBSIDs:', outlier_obs)
    print('Unique observation names:', len(obsdata))
    print()
    return obsdata


In [3]:
# Define variables

# directory of dumped EW csv files
# dump_ew_dir = '/Users/mlessard/REU_2022/data/dump_ew_csv/'
dump_ew_dir = '/seti/src/f-ring/data_files/'

#use all 3 regions, 4 boundaries between the regions
radii = ['139470', '140170', '140265', '140965']

size = '1' #slice/step size of the csv file, currently only have 0

#degrees within 90 (incidence angle) to cut off/get rid of observations
eqx_cutoff = 1.5

#list outlier obsids
outliers = ['ISS_091RF_FMOVIE003_PRIME', 'ISS_107RF_FMOVIE002_PRIME', 'ISS_189RF_FMOVIE001_PRIME']

In [4]:
# Read in the CSV files (full-width, then the 3 different regions)

#get the full-width image data
full_img_filepath = f'{dump_ew_dir}ew_stats_{radii[0]}_{radii[1]}_{radii[2]}_{radii[3]}_{size}.csv'
full_orig_obsdata = pd.read_csv(full_img_filepath, parse_dates=['Date'], index_col='Observation'); ms=20; alpha=0.7

#calculate mu, mu0 
full_orig_obsdata['Mu'] = np.abs(np.cos(np.radians(full_orig_obsdata['Mean Emission'])))
full_orig_obsdata['Mu0'] = np.abs(np.cos(np.radians(full_orig_obsdata['Incidence'])))

In [5]:
#print initial statistics
print('** SUMMARY STATISTICS **')
print('Unique observation names:', len(full_orig_obsdata.groupby('Observation')))
print('Total slices:', len(full_orig_obsdata))
print('Starting date:', full_orig_obsdata['Date'].min())
print('Ending date:', full_orig_obsdata['Date'].max())
print('Time span:', full_orig_obsdata['Date'].max()-full_orig_obsdata['Date'].min())
print()

print(full_img_filepath)
print()


** SUMMARY STATISTICS **
Unique observation names: 1
Total slices: 194
Starting date: 2009-05-26 09:08:14
Ending date: 2009-05-26 16:42:38
Time span: 0 days 07:34:24

/seti/src/f-ring/data_files/ew_stats_139470_140170_140265_140965_1.csv



In [6]:
obs=full_orig_obsdata
corr = compute_corrected_ew_col(obs, col_tau=('Normal EWI', None, 'Normal EWC', 0.042, 'Normal EWO', None))
diff = obs['Normal EW3Z'] - corr
print(diff.min(), diff.max(), diff.mean())

-0.00015348647584034758 0.00012612606074924315 3.02485564074993e-06


In [7]:
diff==diff.max()

Observation
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111RF_FMOVIE002_PRIME    False
ISS_111R